In [ ]:
import requests
import json

import pandas as pd

headers1 = {
    # Request headers of fund daily info
    'Ocp-Apim-Subscription-Key': 'c1714d1753d5947f5b8f6f3cb67354d1d'
}

headers2 = {
    # Request headers of fund fact sheet
    'Ocp-Apim-Subscription-Key': '89cbe200e6ee4afca33f6e2122aa5e82'
}

In [ ]:
response = requests.get(f'https://api.sec.or.th/FundFactsheet/fund/amc/C0000000239', headers = headers2)
status = json.loads(response.text)
status_df = pd.DataFrame(status)
status_df_use = status_df[['proj_id','cancel_date','fund_status','proj_abbr_name','proj_name_en']]
status_df_use

In [ ]:
import pandas as pd
SCB_amc_data = pd.DataFrame(columns = ['proj_id','risk_spectrum','spec_code','amc','name_th','proj_retail_type'])
file = pd.read_csv('./amc_data/SCB_amc_data/SCB_AMC_DATA.csv')
file['amc'] =  'C0000000239'
file['name_th'] = 'บริษัท หลักทรัพย์จัดการกองทุนไทยพาณิชย์ จำกัด'
SCB_amc_data = pd.concat([SCB_amc_data,file])
SCB_amc_data_clean = SCB_amc_data.dropna(subset=['spec_code','proj_retail_type'])
SCB_amc_data_clean.drop(SCB_amc_data_clean[SCB_amc_data_clean['proj_retail_type'] != 'R'].index, inplace = True)
SCB_amc_no_rmf_ssf = SCB_amc_data_clean[~(SCB_amc_data_clean['spec_code'].str.contains('RMF', case=False, na=False) | SCB_amc_data_clean['spec_code'].str.contains('SSF', case=False, na=False))]
len(SCB_amc_no_rmf_ssf)

In [ ]:
SCB_amc_no_rmf_ssf = SCB_amc_no_rmf_ssf.reset_index()
SCB_amc_no_rmf_ssf.drop(columns=['index'])
SCB_amc_no_rmf_ssf

In [ ]:
SCB_amc_no_rmf_ssf = pd.merge(SCB_amc_no_rmf_ssf, status_df_use, on='proj_id')
SCB_amc_no_rmf_ssf

In [ ]:
SCB_amc_no_rmf_ssf = SCB_amc_no_rmf_ssf.drop(SCB_amc_no_rmf_ssf[SCB_amc_no_rmf_ssf["cancel_date"]!="-"].index)
SCB_amc_no_rmf_ssf = SCB_amc_no_rmf_ssf.drop(SCB_amc_no_rmf_ssf[SCB_amc_no_rmf_ssf["fund_status"]=="EX"].index)
SCB_amc_no_rmf_ssf

In [ ]:
SCB_amc_no_rmf_ssf = SCB_amc_no_rmf_ssf.reset_index()

In [ ]:
SCB_amc_no_rmf_ssf = SCB_amc_no_rmf_ssf.drop(columns=['level_0','index'])
SCB_amc_no_rmf_ssf

In [ ]:
import re
import pdfbox
import os
from IPython.display import clear_output
p = pdfbox.PDFBox()
not_screenable = []
max_len = len(SCB_amc_no_rmf_ssf)
for index, row in SCB_amc_no_rmf_ssf.iterrows():
        response = requests.get(f'https://api.sec.or.th/FundFactsheet/fund/{row["proj_id"]}/URLs',headers=headers2)
        res_text = json.loads(response.text)
        res_pdf = requests.get(res_text['url_factsheet'])
        with open(f"tmp/tmp.pdf", 'wb') as pdf_file:
                pdf_file.write(res_pdf.content)
        try:
                print(str(index+1) + ' of ' + str(max_len) +' '+ str(round(((index+1)/max_len)*100,2))+' % Completed' )
                if os.path.getsize("tmp/tmp.pdf") < 216:
                        raise Exception("File not found ::") 
                p.extract_text(f'tmp/tmp.pdf')
                with open(f'tmp/tmp.txt', 'r',  encoding="utf8") as file:
                        data = file.read()
                match1 = re.search(r'(\S+)\s*:\s*เสี่ยง', data)
                match2 = re.search(r'(\S+)\s*:\s*เสียง', data)
                match3 = re.search(r'(\S+)\s*:\s*เสยง', data)
                if match1:
                        number_before_string = match1.group(1)
                        SCB_amc_no_rmf_ssf.at[index, 'risk_spectrum'] = number_before_string
                elif match2:
                        number_before_string = match2.group(1)
                        SCB_amc_no_rmf_ssf.at[index, 'risk_spectrum'] = number_before_string
                elif match3:
                        number_before_string = match3.group(1)
                        SCB_amc_no_rmf_ssf.at[index, 'risk_spectrum'] = number_before_string
                else:
                        print("Pattern not found on "+ row["proj_id"])
                        not_screenable.append(row["proj_id"])
                clear_output(wait=True)
        except Exception as e:
                print(str(e) +' Error found on '+ row['proj_id'])
                

In [ ]:
not_screenable

In [ ]:
for proj_id in not_screenable:
        response = requests.get(f'https://api.sec.or.th/FundFactsheet/fund/{proj_id}/URLs',headers=headers2)
        res_text = json.loads(response.text)
        res_pdf = requests.get(res_text['url_factsheet'])
        with open(f"tmp/{proj_id}.pdf", 'wb') as pdf_file:
                pdf_file.write(res_pdf.content)

In [ ]:
SCB_amc_no_rmf_ssf

In [ ]:
SCB_amc_no_rmf_ssf_nan = SCB_amc_no_rmf_ssf[SCB_amc_no_rmf_ssf.isna().any(axis=1)]
SCB_amc_no_rmf_ssf_nan

In [ ]:
SCB_amc_no_rmf_ssf = SCB_amc_no_rmf_ssf.dropna()
SCB_amc_no_rmf_ssf

In [ ]:
SCB_amc_no_rmf_ssf_nan

In [ ]:
#read manually
risk_list = ['8','3','3','3','8','3','3','8']
SCB_amc_no_rmf_ssf_nan.loc[SCB_amc_no_rmf_ssf_nan['risk_spectrum'].isna(), 'risk_spectrum'] = risk_list
SCB_amc_no_rmf_ssf_nan

In [ ]:
result_SCB_amc_no_rmf_ssf = pd.concat([SCB_amc_no_rmf_ssf, SCB_amc_no_rmf_ssf_nan], ignore_index=True)

In [ ]:
result_SCB_amc_no_rmf_ssf

In [ ]:
result_SCB_amc_no_rmf_ssf['proj_id'].nunique()

In [ ]:
result_SCB_amc_no_rmf_ssf.to_csv('../Result_amc/SCB/SCB_NON_RMF_SSF.csv', encoding='utf8')

In [29]:
df = pd.read_csv('../Result_amc/SCB/SCB_NON_RMF_SSF.csv', encoding='utf8')
df

,Unnamed: 0,proj_id,risk_spectrum,spec_code,amc,name_th,proj_retail_type,cancel_date,fund_status,proj_abbr_name,proj_name_en
0,0,M0004_2564,5,FED,C0000000239,บริษัท หลักทรัพย์จัดการกองทุนไทยพาณิชย์ จำกัด,R,-,RG,SCBAA,SCB Active Allocation Fund
1,1,M0015_2564,6,FED,C0000000239,บริษัท หลักทรัพย์จัดการกองทุนไทยพาณิชย์ จำกัด,R,-,RG,SCBAO,SCB Asia Opportunity Fund
2,2,M0024_2560,6,FED,C0000000239,บริษัท หลักทรัพย์จัดการกองทุนไทยพาณิชย์ จำกัด,R,-,RG,SCBUSHY,SCB US Short Duration High Income Bond
3,3,M0026_2539,6,IND,C0000000239,บริษัท หลักทรัพย์จัดการกองทุนไทยพาณิชย์ จำกัด,R,-,RG,SCBSETFUND,SCB SET INDEX FUND
4,4,M0026_2551,6,FED,C0000000239,บริษัท หลักทรัพย์จัดการกองทุนไทยพาณิชย์ จำกัด,R,-,RG,SCBAEMFUND,SCB ASIAN EMERGING MARKETS OPEN END FUND
...,...,...,...,...,...,...,...,...,...,...,...
90,90,M0511_2566,3,CRF,C0000000239,บริษัท หลักทรัพย์จัดการกองทุนไทยพาณิชย์ จำกัด,R,-,RG,SCBCP6M15,SCB Capital Protection 6M15
91,91,M0522_2555,8,FED,C0000000239,บริษัท หลักทรัพย์จัดการกองทุนไทยพาณิชย์ จำกัด,R,-,RG,SCBOILFUND,SCB OIL FUND
92,92,M0549_2566,3,CRF,C0000000239,บริษัท หลักทรัพย์จัดการกองทุนไทยพาณิชย์ จำกัด,R,-,RG,SCBCP6M16,SCB Capital Protection 6M16
93,93,M0570_2566,3,CRF,C0000000239,บริษัท หลักทรัพย์จัดการกองทุนไทยพาณิชย์ จำกัด,R,-,RG,SCBCP6M17,SCB Capital Protection 6M17


In [30]:
df['risk_spectrum'].value_counts()

risk_spectrum
3    47
6    26
5    13
7     4
8     3
1     2
Name: count, dtype: int64

### SE = อนุมัติ(อยู่ระหว่าง Filing) 